In [1]:
import numpy as np
import pandas as pd
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
from sklearn.decomposition import PCA
import random
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.utils import shuffle

# All Features

In [2]:
genres = pd.read_csv('fma_metadata/tracks.csv', low_memory=False).dropna(subset=['track.7'])
genres = genres.rename(index=str, columns={'Unnamed: 0': 'id', 'track.7': 'genre'})
genres = genres[['id', 'genre']]
genres = genres[1:]

features = pd.read_csv('fma_metadata/features.csv', low_memory=False)
features = features.rename(index=str, columns={'feature': 'id'})
features = features[1:]

features = features.merge(genres, on='id')
features.head()

,id,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6,genre
0,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,...,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02,Hip-Hop
1,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,...,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02,Hip-Hop
2,5,5.2756297588e-01,-7.7654317021e-02,-2.7961030602e-01,6.8588310480e-01,1.9375696182e+00,8.8083887100e-01,-9.2319184542e-01,-9.2723226547e-01,6.6661673784e-01,...,1.2690781616e-02,1.4759079553e-02,6.8084154129e+00,3.7500000000e-01,5.3114086390e-02,4.1503906250e-02,0.0000000000e+00,2.1933031082e+00,4.4860601425e-02,Hip-Hop
3,10,3.7022454739e+00,-2.9119303823e-01,2.1967420578e+00,-2.3444947600e-01,1.3673638105e+00,9.9841135740e-01,1.7706941366e+00,1.6045658588e+00,5.2121698856e-01,...,1.7951935530e-02,1.3921394013e-02,2.1434211731e+01,4.5214843750e-01,7.7514506876e-02,7.1777343750e-02,0.0000000000e+00,3.5423245430e+00,4.0800448507e-02,Pop
4,134,9.1844475269e-01,6.7414724827e-01,5.7781827450e-01,1.2811170816e+00,9.3374562263e-01,7.8176945448e-02,1.1992042065e+00,-1.7522314191e-01,9.2548191547e-01,...,1.6322381794e-02,1.5819497406e-02,4.7310867310e+00,4.1943359375e-01,6.4369551837e-02,5.0781250000e-02,0.0000000000e+00,1.8061059713e+00,5.4622855037e-02,Hip-Hop


In [3]:
data = features.sample(25000)

training_set = data[:19922]
validation_set = data[19922:22427]
test_set = data[22427:]

enc = LabelEncoder()
y_train = enc.fit_transform(training_set['genre'])
y_val = enc.transform(validation_set['genre'].values)
y_test = enc.transform(test_set['genre'].values)

scaler = StandardScaler(copy=False)

data = data.drop('genre', axis=1)

training_set = data[:19922]
validation_set = data[19922:22427]
test_set = data[22427:]

X_train = training_set.as_matrix()
X_val = validation_set.as_matrix()
X_test = test_set.as_matrix()

scaler.fit_transform(X_train)
scaler.transform(X_val)
scaler.transform(X_test)

C:\Users\Daiyaan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Daiyaan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Daiyaan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Daiyaan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.17484855e+00,  3.92205278e-02,  3.29844360e-01, ...,
        -4.91687495e-01, -8.82104808e-01, -6.78403548e-01],
       [ 1.41900109e+00, -1.20246905e-01, -6.43709126e-02, ...,
        -4.91687495e-01, -8.56998456e-01, -1.04822339e+00],
       [-4.60494985e-01, -8.13729202e-02, -5.27365307e-02, ...,
        -1.28962138e-01,  9.33860194e-01, -2.30803373e-01],
       ...,
       [-1.44993421e-01, -9.23905387e-02, -7.93337293e-02, ...,
         2.22875268e+00,  1.08314730e+00, -1.01384874e+00],
       [-3.92752091e-01, -4.33050514e-02,  5.74505532e-04, ...,
        -4.91687495e-01,  3.34326864e-01,  2.49769153e-01],
       [-1.48450831e+00, -1.16816053e-01, -1.27333837e-01, ...,
         5.24005410e-02,  1.11014976e+00, -5.21418404e-01]])

In [4]:
clf = SVC(C=1.5, gamma='auto')
clf.fit(X_train, y_train) 

clf.score(X_test, y_test), clf.score(X_train, y_train)

(0.27788573649436454, 1.0)

# PCA Reduced Features

In [2]:
genres = pd.read_csv('fma_metadata/tracks.csv', low_memory=False).dropna(subset=['track.7'])
genres = genres.rename(index=str, columns={'Unnamed: 0': 'id', 'track.7': 'genre'})
genres = genres[['id', 'genre']]
genres = genres[1:]

features = pd.read_csv('fma_metadata/features.csv', low_memory=False)
features = features.rename(index=str, columns={'feature': 'id'})
features = features[1:]

features = features.merge(genres, on='id')
features.head()

,id,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6,genre
0,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,...,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02,Hip-Hop
1,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,...,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02,Hip-Hop
2,5,5.2756297588e-01,-7.7654317021e-02,-2.7961030602e-01,6.8588310480e-01,1.9375696182e+00,8.8083887100e-01,-9.2319184542e-01,-9.2723226547e-01,6.6661673784e-01,...,1.2690781616e-02,1.4759079553e-02,6.8084154129e+00,3.7500000000e-01,5.3114086390e-02,4.1503906250e-02,0.0000000000e+00,2.1933031082e+00,4.4860601425e-02,Hip-Hop
3,10,3.7022454739e+00,-2.9119303823e-01,2.1967420578e+00,-2.3444947600e-01,1.3673638105e+00,9.9841135740e-01,1.7706941366e+00,1.6045658588e+00,5.2121698856e-01,...,1.7951935530e-02,1.3921394013e-02,2.1434211731e+01,4.5214843750e-01,7.7514506876e-02,7.1777343750e-02,0.0000000000e+00,3.5423245430e+00,4.0800448507e-02,Pop
4,134,9.1844475269e-01,6.7414724827e-01,5.7781827450e-01,1.2811170816e+00,9.3374562263e-01,7.8176945448e-02,1.1992042065e+00,-1.7522314191e-01,9.2548191547e-01,...,1.6322381794e-02,1.5819497406e-02,4.7310867310e+00,4.1943359375e-01,6.4369551837e-02,5.0781250000e-02,0.0000000000e+00,1.8061059713e+00,5.4622855037e-02,Hip-Hop


In [3]:
data = features.sample(25000)

training_set = data[:19922]
validation_set = data[19922:22427]
test_set = data[22427:]

enc = LabelEncoder()
y_train = enc.fit_transform(training_set['genre'])
y_val = enc.transform(validation_set['genre'].values)
y_test = enc.transform(test_set['genre'].values)

scaler = StandardScaler(copy=False)

pca = PCA(n_components=275)
pca.fit(data.drop('genre', axis=1).as_matrix().T)
data = pd.DataFrame(pca.components_.T)

training_set = data[:19922]
validation_set = data[19922:22427]
test_set = data[22427:]

X_train = training_set.as_matrix()
X_val = validation_set.as_matrix()
X_test = test_set.as_matrix()

scaler.fit_transform(X_train)
scaler.transform(X_val)
scaler.transform(X_test)

array([[-1.55977573,  1.51909576, -0.29091038, ..., -1.03029923,
        -0.83162177,  0.53587042],
       [ 0.02166803, -0.13373329, -0.34006884, ...,  0.21611512,
        -1.85053953, -0.416559  ],
       [-0.2856438 ,  0.12723044, -1.17623878, ..., -0.32664934,
        -0.60598507, -0.18088152],
       ...,
       [-1.22440888,  1.1130143 , -1.03924643, ...,  1.14719908,
        -0.12780583,  1.26973126],
       [ 0.55109119, -0.55012009,  0.34411873, ...,  0.89581704,
         0.12089336,  1.15717321],
       [-0.95215279,  0.80248139, -1.21813032, ...,  1.0174678 ,
        -0.25658993,  0.10892382]])

In [4]:
clf = SVC(C=1.5, gamma='auto')
clf.fit(X_train, y_train) 

clf.score(X_test, y_test), clf.score(X_train, y_train)

(0.6583754372328022, 0.8857544423250677)